# PyTorch Fashion MNIST example

"Hello world" type example, based on https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import json
import pickle
import time
from pathlib import Path

import torch
import torch.nn as nn
import torchvision
from dotmap import DotMap
from magic_timer import MagicTimer
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
# print versions of imported libraries if can
import sys

for module_name in sorted(set(sys.modules) & set(globals())):
    if version := getattr(sys.modules[module_name], "__version__", None):
        print(module_name, version)

json 2.0.9
torch 1.12.1
torchvision 0.13.1
tqdm 4.64.1


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
get_timestamp = lambda: time.strftime("%Y%m%d-%H%M%S")
start_time = get_timestamp()
cf = DotMap(
    {
        # training config
        "batch_size": 64,
        "learning_rate": 1e-3,
        "num_epochs": 10,
        "device": device,
        # meta config
        "ckpt_dir": f"data/PyTorch_Fashion_MNIST_example_ckpts/{start_time}",
        "notebook_start_time": start_time,
        "save_every_secs": 30,
        # active content
        "last_saved_time": None,
        "model": None,
        "optimizer": None,
        "epoch_idx": None,
        "batch_idx": None,
    }
)

In [4]:
class ConfigEncoder(json.JSONEncoder):
    "Handle things in `cf` that Pickle can't dump."

    def default(self, obj):
        if isinstance(obj, nn.Module):
            return str(obj)
        elif isinstance(obj, torch.optim.Optimizer):
            return str(obj)
        return json.JSONEncoder.default(self, obj)


print(json.dumps(cf.toDict(), cls=ConfigEncoder, indent=2))

{
  "batch_size": 64,
  "learning_rate": 0.001,
  "num_epochs": 10,
  "device": "cuda",
  "ckpt_dir": "data/PyTorch_Fashion_MNIST_example_ckpts/20220919-185117",
  "notebook_start_time": "20220919-185117",
  "save_every_secs": 30,
  "last_saved_time": null,
  "model": null,
  "optimizer": null,
  "epoch_idx": null,
  "batch_idx": null
}


In [5]:
def save():
    cf.current_save_time = time.time()
    ts = get_timestamp()
    Path(cf.ckpt_dir).mkdir(exist_ok=True, parents=True)
    with open(f"{cf.ckpt_dir}/{ts}.pickle", "wb") as f:
        pickle.dump(cf, f)
    with open(f"{cf.ckpt_dir}/{ts}.json", "w") as f:
        json.dump(cf.toDict(), fp=f, cls=ConfigEncoder, indent=2)
    cf.last_saved_time = time.time()

In [6]:
training_data = torchvision.datasets.FashionMNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)
test_data = torchvision.datasets.FashionMNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=cf.batch_size)
test_dataloader = DataLoader(test_data, batch_size=cf.batch_size)

In [7]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}")
    for batch_idx, (X, y) in enumerate(pbar):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})
        # Keep track of these globally
        cf.epoch_idx = epoch
        cf.batch_idx = batch_idx
        if time.time() - cf.last_saved_time > cf.save_every_secs:
            print("Saving model.")
            save()


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test set: Accuracy = {(100*correct):.1f}%, Mean loss = {test_loss:.4f}")

In [8]:
# Train for `cf.num_epochs` epochs
timer = MagicTimer()
model = Model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=cf.learning_rate)
loss_fn = nn.CrossEntropyLoss().to(device)
# Checkpoint these
cf.model = model
cf.optimizer = optimizer
# Save the initial model
save()
for i in range(1, cf.num_epochs + 1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch=i)
    test_loop(test_dataloader, model, loss_fn)
save()
print(f"Trained model in {timer}.")

Epoch 1: 100%|██████████| 938/938 [00:09<00:00, 97.82it/s, loss=2.1461] 


Test set: Accuracy = 53.0%, Mean loss = 2.1478


Epoch 2: 100%|██████████| 938/938 [00:08<00:00, 105.74it/s, loss=1.8790]


Test set: Accuracy = 59.1%, Mean loss = 1.8677


Epoch 3: 100%|██████████| 938/938 [00:08<00:00, 106.79it/s, loss=1.5491]


...

Test set: Accuracy = 69.2%, Mean loss = 0.8105


Epoch 10:  16%|█▌        | 150/938 [00:01<00:07, 107.29it/s, loss=0.7341]

Saving model.


Epoch 10: 100%|██████████| 938/938 [00:08<00:00, 105.70it/s, loss=0.8850]


Test set: Accuracy = 70.4%, Mean loss = 0.7805
Trained model in 1.7 minutes.
